**Can be ignored.**


In [ ]:
from utils import get_tickers, download_m1_raw_data
from datetime import date
from polygon.rest import RESTClient
import pandas as pd
import os
import pyarrow.parquet as pq

DATA_PATH = "../data/polygon/"

with open(DATA_PATH + "secret.txt") as f:
    KEY = next(f).strip()

client = RESTClient(api_key=KEY)

Some fastparquet files were corrupt for unknown reasons. So I download them again.

In [ ]:
tickers = get_tickers(3)

corrupt = []
files = os.listdir(DATA_PATH + f"raw/m1/")
for file in files:
    try:
        pd.read_parquet(DATA_PATH + f"raw/m1/{file}")
    except:
        corrupt.append(file)
        print(file)

In [ ]:
for file in corrupt:
    id = file[:-8]
    ticker = id[:-11]
    start = tickers.loc[tickers['ID'] == id, 'start_date'].values[0]
    end = tickers.loc[tickers['ID'] == id, 'end_date'].values[0]    
    m1 = download_m1_raw_data(ticker = ticker, from_ = start, to = end, columns = ["open", "high", "low", "close", "volume"], client=client)
    m1.to_parquet(DATA_PATH + f"raw/m1/{id}.parquet", engine="fastparquet", compression="snappy", row_group_offsets=25000)
    print(id)